# Analytic Hierarchy Process 层次分析法 (AHP)
步骤
- 解决评价问题
- 画出层级结构图
- 构造准则 指标重要矩阵
- 依照评价指标对各个方案打分
- 求出权重 填表 求最后得分
- 层次总排序一致性检验

目标: 选择大学工程系

准则: 
- 成本
- 学术排名
- 校外课程

可选方案:
- A大
- B大
- C大

正互反矩阵:
$a_{ij} > 0, a_{ij}\times a_{ji} = 1$ 则该矩阵为正互反矩阵
$\frac{C_i}{C_j} \times \frac{C_j}{C_i} = 1$

从矩阵上看 左上对角到右下对角为1 且 右上区域 和 左下区域互为倒数

In [15]:
import numpy as np
from scipy.linalg import eig

# 正互反矩阵 指标重要性
A = np.array([
    [1, 5, 3],  # 成本
    [1/5, 1, 3],  # 学术排名
    [1/3, 1/3, 1]  # 校外课程
])

# 定义三个准则的正互反矩阵
cost_matrix = np.array([
    [1, 1/2, 1/3],  # A大
    [2, 1, 1/2],    # B大
    [3, 2, 1]       # C大
])

academic_matrix = np.array([
    [1, 3, 5],  # A大
    [1/3, 1, 2],  # B大
    [1/5, 1/2, 1]  # C大
])

extracurricular_matrix = np.array([
    [1, 1/4, 1/7],  # A大
    [4, 1, 1/3],    # B大
    [7, 3, 1]       # C大
])


特征根（也称为特征值）
- 线性代数概念
- 与矩阵相关

矩阵 $A$ 与向量 $v$
满足 $A \cdot v = \lambda \cdot v \\$
 $则 \lambda$ 为矩阵 \(A\) 的一特征值（特征根）
 ，而则为 \(v\) 对应的特征向量。

特征根为一标量，
它表示矩阵在某个方向上的伸缩比例。
特征向量为此方向上的向量。

举例
有一 2x2 矩阵：
$ A = \begin{pmatrix} 4 & 1 \\ 2 & 3 \end{pmatrix} $

想找其特征值 $\lambda$，则需解此特征方程：
$ \text{det}(A - \lambda I) = 0 $

其中，$I$ 为单位矩阵，$\text{det}$ 表示行列式。

对上述矩阵 \(A\)，特征方程为：
$\begin{vmatrix} 4 - \lambda & 1 \\ 2 & 3 - \lambda \end{vmatrix} = 0 $

解这个方程可得特征值 $\lambda$。



In [16]:
# 计算特征值和特征向量
def calculate_eig(matrix):
    eigvals, eigvecs = eig(matrix)
    return eigvals, eigvecs




eigvals, eigvec = calculate_eig(A)

print("特征值:", eigvals)
print("特征向量:", eigvec)


特征值: [ 3.29477949+0.j         -0.14738975+0.97442788j -0.14738975-0.97442788j]
特征向量: [[-0.93050025+0.j          0.93050025+0.j          0.93050025-0.j        ]
 [-0.31822661+0.j         -0.15911331+0.27559233j -0.15911331-0.27559233j]
 [-0.18138662+0.j         -0.09069331-0.15708542j -0.09069331+0.15708542j]]


需确定各方案与准则层所评出的矩阵为一致性矩阵
（也就是评分是不是一致 会不会前后矛盾）

一致性判断方法：
- 正互反矩阵A的秩为1,A的唯一非零特征根为n;
- 正互反矩阵A的任一列向量都对特征根n的特征向量;
- 当正互反矩阵A不为一致矩阵，其最大特征根 $\lambda_{max} > n. 相差越大越不一致$

# 一致性检验方法 
定义一致性指标CI(Consistency Index)   


$CI=\frac{\lambda_{max}-n}{n-1} , CI = \begin{cases}
0， \text{有完全一致性}\\
\to 0， \text{满意一致性}\\
\to \infty， \text{越大越差}\\
\end{cases}$  

为衡量CI大小 引入随机一致性RI (Random index)  先构造m个判断矩阵
$ \\RI = \frac{CI_1+CI_2+...+CI_m}{m} = \frac{\frac{\lambda_1+\lambda_2+...+\lambda_m}{m}-n}{n-1}  $

定义一致性比例 $CR=\frac{CI}{RI}$ 
若CR<0.1 则一致性可接受 Else ≥ 修正  

如何修正?  
强行往一致性靠，调整成倍数关系即可


In [19]:

# 对准则指标重要性进行一致性验证

def calculate_consistency_ratio(matrix):
    eigvals, eigvecs = eig(matrix)
    lambda_max = np.max(eigvals).real # 最大特征值
    n = matrix.shape[0] # 矩阵的阶数
    CI = (lambda_max - n) / (n - 1) # 一致性指标
    RI_dict = {1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
    RI = RI_dict.get(n, 1.49) # 随机一致性指标
    CR = CI / RI # 一致性比例
    return CR

def adjust_to_consistency(matrix):
    n = matrix.shape[0] # 矩阵的阶数
    geometric_means = np.zeros(n) # 几何平均值
    
    # 计算每一列的几何平均值
    for i in range(n): 
        geometric_means[i] = np.prod(matrix[:, i]) ** (1/n) 
    
    # 归一化几何平均值
    normalized_geometric_means = geometric_means / np.sum(geometric_means)
    
    # 用归一化后的几何平均值构造新的矩阵
    consistent_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            consistent_matrix[i, j] = normalized_geometric_means[i] / normalized_geometric_means[j]
    
    return consistent_matrix

R = calculate_consistency_ratio(A)
print(f"初始一致性比例CR: {CR}")

# 如果一致性未通过，进行修正
if CR >= 0.1:
    A = adjust_to_consistency(A)
    CR = calculate_consistency_ratio(A)
    print(f"调整后的一致性比例CR: {CR}")

print("一致性通过")
print("修正后的矩阵:")
print(A)

初始一致性比例CR: 0.2541202537235106
调整后的一致性比例CR: 7.656710514656253e-16
一致性通过
修正后的矩阵:
[[1.         0.34199519 0.19493452]
 [2.92401774 1.         0.56999198]
 [5.12992784 1.75441064 1.        ]]


# 一致性矩阵权重计算

归一化值为一致性矩阵权重

什么是归一化?  
归一化为令矩阵元素进行标准化处理，满足特定条件，将其转换成一个标准范围内（纯粹方便处理）

举例理解:  
LLM 大语言模型简单来说一般是预测下一个字的概率，在最后输出时需使用softmax(归一化函数)将output局限在0到1之间

AHP归一化怎么做?  
将特征向量的各个元素进行归一化处理，使其和为1。
归一化的方法是将每个元素除以特征向量的所有元素之和。

In [20]:
# 归一化特征向量作为权重

eigvals, eigvecs = eig(A)

max_eigval_index = np.argmax(eigvals) # 最大特征值的索引
max_eigvec = eigvecs[:, max_eigval_index].real   

# 归一化特征向量，得到权重
weights = max_eigvec / np.sum(max_eigvec)

print("准则的权重:")
print(weights)

准则的权重:
[0.11044908 0.32295508 0.56659583]


# 非一致性矩阵 -算术平均法求权重


举例：

$
A = \begin{pmatrix}
1 & 2 & 4 \\
0.5 & 1 & 3 \\
0.25 & \frac{1}{3} & 1
\end{pmatrix}
$

举例算第一列(y轴) 详细算法为:
$\frac{格的数值}{整列总数}$  
$\frac{1}{1+0.5+0.25}$

论文写法:
$对于判断矩阵A = 
A = \begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
，先将其归一化，再将归一化的矩阵按列相加，并将每个元素除以n得到权重向量，即\omega_i = \frac{1}{n}\sum^n_{j=1}\frac{a_{ij}}{\sum^n_{k=1}a_{kj}}(i =1,2,...,n)$

In [ ]:
def arithmetic_mean_weights(matrix):

    column_sums = np.sum(matrix, axis=0) # 每一列的和
    normalized_matrix = matrix / column_sums # 归一化矩阵
    

    weights = np.mean(normalized_matrix, axis=1) # 求每一行的均值
    
    return weights


weights_arithmetic_mean = arithmetic_mean_weights(A) # 求权重
print("非一致性矩阵的算术平均法求权重:")
print(weights_arithmetic_mean)

# 特征值求权重

$矩阵A \to^{计算特征值与特征向量}\lambda_1 ... \to^{一致性检验} CI <0.1, 通过后\to^{最大特征值对应的特征向量} \alpha = 
\begin{pmatrix}
-0.93050025 \\
-0.31822661 \\
-0.18138662
\end{pmatrix}    \to^{归一化} \omega =    \begin{pmatrix}
w_1 \\
w_2 \\
w_3
\end{pmatrix}           $

两种求法的区别  
特征值求权重 （常用方便）  
算术平均法 （二者共用 取平均值）  

In [ ]:
# 计算特征值和特征向量
eigvals, eigvecs = eig(A)

# 找到最大特征值对应的特征向量
max_eigval_index = np.argmax(eigvals)
max_eigvec = eigvecs[:, max_eigval_index].real

# 归一化特征向量，得到权重
weights = max_eigvec / np.sum(max_eigvec)

print("特征值求权重:")
print(weights)